In [ ]:
from drive.google_main import Google
from drive.error import Error

class file(Google):
    def __init__(self,file_name) -> None:
        super().__init__()
        # puxando nome do arquivo que será substituido
        self.__file = file_name
        self.service = self.generate_service()

    def generate_service(self):
        #Keys para capturar pastas drive
        SCOPES = ['https://www.googleapis.com/auth/drive.file']
        CLIENT_SECRET_FILE = self.chromedriver
        API_NAME = 'drive'
        API_VERSION = 'V3'
        return self.Create_Service(CLIENT_SECRET_FILE,API_NAME,API_VERSION,SCOPES)

    def move_file(self,
                  folder_id=str,
                  folder_target=str,
                  returnblob=False):
        '''
        To change a file to another folder first of all we need to find you file and to do 
        this the function has differents methods:
        \nFind item by name -> To find an item by name, you can use the function like this:\n
        >>> obj = file('namefile')
        >>> obj.move_file('name',folder_id,folder_target)\n

        The function has the method return blob, if you call this by True, the function will return the item blob in drive.
        '''
        self.__id = 143509
        # checando se realmente estamos recebendo o nome str de um arquivo
        if type(self.__file) == str:
            # query para filtrar a pasta pai e o nome do arquivo target
            query = f"parents = '{folder_id}' and name = '{self.__file}'"
            # localizando file, pelo correto irá retornar apenas um item
            file = self.service.files().list(q=query).execute()
            print(file)
            # checando se localizamos um item e não uma pasta
            if file['files'][0]['mimeType'] == 'application/vnd.google-apps.folder':
                # se sim enviamos a mensagem de erro
                Error(self.__id,'folder_not_file',file['files']['mimeType'])
                return
            else:
                # se não podemos seguir com a modificação
                self.service.files().update(
                    fileId       = file['files'][0]['id'],
                    addParents    = folder_target,
                    removeParents = folder_id
                ).execute()
            if returnblob==True:
                return file


    def move_files(self,folder_id):
        '''
            For this method the code expect to receive a list or a dic with the itens,\n
                if you want to send all the itens to the same folder, you can use the function like:\n
                >>> objt = file([file1,file2,folder_dest])
                >>> move_files(folder_id)
                \n
                but if you want to send each item for a diferent folder, you can use the function like:\n
                >>> objt = file({file1:folderkey,file2:folderkey})
                >>> move_files(folder_id)
                \n
                the function expect to receive all folder keys to each item.
        '''
        self.__id=143510
        # proceso para lista de itens
        if type(self.__file) == list:
            # chechando se a lista esta correta 
            if len(self.__file)<=1:
                Error(self.__id,"len<1",self.__file)
                return
            for file in self.__file[0:-1]:
                query = f"parents = '{folder_id}' and name = '{file}'"
                response = self.service.files().list(q=query).execute()
                print(file)
        # procesos para itens em diferente pastas
        elif type(self.__file) == dict:
            for name_item in self.__file:
                key_dest = self.__file[name_item]
                print(key_dest)
# teste=file({'1197769':"tes",'1197768':'pá'})